In [1]:
import camelot
import pandas as pd
import numpy as np
import pdfminer as pm
import math
from datetime import datetime  
from datetime import timedelta  
import warnings
import os
import subprocess
import ocrmypdf
import sys
import traceback

In [2]:
## Read in all table regions
def convertPDFToTable(filepath):
    table_cols = [['62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573',
                  '62, 92, 114, 154, 194, 287, 311, 384, 414, 524, 573'], 
                  ['44,']]
    curColSetting = 0
    p1t = camelot.read_pdf(filepath, 
                         strip_text='.\n', flavor='stream', pages="1-end",
                         #table_regions=['0,700,600,0'],
                          columns=table_cols[curColSetting], 
                           edge_tol=1000)
    pdf = []
    for t in p1t:
        pdf.append(t.df)
    return pdf

In [3]:
def processInvoice(tables, committee, station):
    ads = pd.DataFrame(columns=['Air Date', 'Air Time', 'Rate', 'Description'])
    def findHeaderRow(row):
        headers =  { 'InventoryCode': ['Air Date', 'Air Time', 'Description', 'Rate'] }
        for htype in headers:
            for val in row:
                try:
                    ind = headers[htype].index(val)
                    headers[htype].remove(val)
                except:
                    pass
        if(len(headers['InventoryCode']) == 0):
            return ['InventoryCode', row.name]
    for t in tables:
        print('table')
        progColName=False
        headerConfig = t.apply(findHeaderRow, axis=1).dropna()
        if len(headerConfig)>0:
            progColName = headerConfig.values[0][0]
        if progColName != False:
            progs = t.copy()
            # Set column headers to be the header row, remove row from data.
            progs.columns = progs.iloc[headerConfig.values[0][1]]
            progs = progs.iloc[headerConfig.values[0][1]:]
            progs = progs.reset_index(drop=True)
            progs = progs.iloc[1:] # Remove residual header row.
            progs = progs.replace('', np.nan, regex=True)
            progs = progs[progs['Air Date']!='Air Date']
            progs = progs.dropna(thresh=len(progs.columns)-1)
            progs = progs[['Air Date', 'Air Time', 'Description', 'Rate']]
            ads = ads.append(progs, ignore_index=True, sort=False)
    ads = ads.drop_duplicates()
    ads['Station'] = station
    ads['Committee'] = committee
    return ads

In [4]:
# Testing loop
folderpath = "/media/andrew/F08C9B848C9B444E/analysis/tv/fccscraper/tests/invoices/"
for item in list(os.scandir(folderpath)):
    print(item.name)
    tbl = convertPDFToTable(folderpath+item.name)
    tbl = processInvoice(tbl, "Test for Arizona", "KAND")
    display(tbl)

kmsb.pdf
table
table


,Air Date,Air Time,Rate,Description,Station,Committee
0,04/29/18,4:25 PM,$4500,4:00 PM-5:00 PM,KAND,Test for Arizona
1,04/29/18,9:08 PM,$22500,9:00 PM-10:00 PM,KAND,Test for Arizona
2,04/28/18,9:12 PM,$17500,Local News @ 9p Sa,KAND,Test for Arizona


kold.pdf
table


,Air Date,Air Time,Rate,Description,Station,Committee
0,11/02/18,5:12 PM,"$1,40000",News 13 at 5p,KAND,Test for Arizona
1,10/31/18,8:48 PM,"$5,00000",Wed 8p-9p,KAND,Test for Arizona


kpho.pdf


KeyboardInterrupt: 